In [3]:
import os

In [4]:
%pwd

'c:\\Users\\User\\OneDrive\\Desktop\\ml-project-with-ml-flow-1\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\User\\OneDrive\\Desktop\\ml-project-with-ml-flow-1'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [8]:
from src.mlProject.constants import *
from src.mlProject.utils.common import *

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [10]:
import os
from src.mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
import logging
import numpy as np

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class DataTransformation:
    def __init__(self, config):
        self.config = config

    def train_test_spliting(self):
        # Load the data
        data = pd.read_csv(self.config.data_path)

        # Transform the target variable and create tenure groups
        data['Churn'] = np.where(data.Churn == 'Yes', 1, 0)
        labels = ["{0} - {1}".format(i, i + 11) for i in range(1, 72, 12)]
        data['tenure_group'] = pd.cut(data.tenure, range(1, 80, 12), right=False, labels=labels)
        
        # Store columns separately
        id_column = data.pop('customerID')
        tenure_column = data.pop('tenure')
        monthly_charge_column = data.pop('MonthlyCharges')
        total_charges_column = data.pop('TotalCharges')

        # Convert to numeric and handle errors
        data['MonthlyCharges'] = pd.to_numeric(monthly_charge_column, errors='coerce')
        data['TotalCharges'] = pd.to_numeric(total_charges_column, errors='coerce')

        # Identify categorical columns
        categorical_columns = data.select_dtypes(include=['object', 'category']).columns

        # Create dummy variables for categorical columns
        data_dummies = pd.get_dummies(data, columns=categorical_columns)

        # Initialize and apply MinMaxScaler
        scaler = MinMaxScaler()

        # Handle non-numeric values by dropping rows with NaN in scaled columns
        data_dummies = data_dummies.dropna(subset=['MonthlyCharges', 'TotalCharges'])

        # Scale all numerical columns
        numeric_columns = ['MonthlyCharges', 'TotalCharges']
        data_dummies[numeric_columns] = scaler.fit_transform(data_dummies[numeric_columns])
        data_dummies = data_dummies.replace({True: 1, False: 0})
        # Split the data into training and test sets (0.75, 0.25 split)
        train, test = train_test_split(data_dummies, test_size=0.25, random_state=42)

        # Save the split datasets to CSV
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        # Log and print shapes of the resulting datasets
        logger.info("Split data into training and test sets")
        logger.info(f"Training set shape: {train.shape}")
        logger.info(f"Test set shape: {test.shape}")

        print(f"Training set shape: {train.shape}")
        print(f"Test set shape: {test.shape}")


In [16]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

INFO:mlProjectLogger:yaml file: config\config.yaml loaded successfully
INFO:mlProjectLogger:yaml file: params.yaml loaded successfully
INFO:mlProjectLogger:yaml file: schema.yaml loaded successfully
INFO:mlProjectLogger:created directory at: artifacts
INFO:mlProjectLogger:created directory at: artifacts/data_transformation
C:\Users\User\AppData\Local\Temp\ipykernel_9488\411764227.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_dummies = data_dummies.replace({True: 1, False: 0})
INFO:__main__:Split data into training and test sets
INFO:__main__:Training set shape: (5274, 51)
INFO:__main__:Test set shape: (1758, 51)


Training set shape: (5274, 51)
Test set shape: (1758, 51)
